In [276]:
import os
import json
import glob


In [277]:

class RawDataReader:
    def __init__(self, file_path):
        with open(file_path, 'r') as file:
            self.data = json.load(file)

        self.id = self.data['ID']
        # print(self.id)
        self.plan_details = self.extract_plan_details()

    def extract_plan_details(self):
        plan_details = []
        for plan in self.data['plans']:
            # print(plan['name'])
            try:
                plan_name = plan['name']
                planning_exam_name = plan['planning_examination']['name']
                planningCT_filename = plan['planning_examination']['dose_DICOM_filename']
                        
                evaluation_exams = []
                for exam in plan['evaluation_examinations']:
                    if exam['completed'] == False:
                            pass
                    else:
                               
                        for opt_case in exam['optimization_cases']:
                            if opt_case.get('completed', True) and 'gradient_descent_result' in opt_case:
                                final_trans_coord = opt_case['gradient_descent_result'].get('final_translation_coordinate', {})
                                x = round(final_trans_coord.get('x', 0), 5)
                                y = round(final_trans_coord.get('y', 0), 5)
                                z = round(final_trans_coord.get('z', 0), 5)
                                exam_detail = {
                                    # 'name': exam['name'],
                                    'final_translation_coordinate': {'x': x, 'y': y, 'z': z},
                                    'repeatedCT_filename': opt_case['gradient_descent_result'].get('dose_DICOM_filename', '')
                                }
                                evaluation_exams.append(exam_detail)
                    
                    
                        plan_details.append({
                            'plan_name': plan_name,
                            'planning_exam_name': planning_exam_name,
                            'planningCT_filename': planningCT_filename,
                            'evaluation_examinations': evaluation_exams
                        })

            except:
                plan_details.append({
                    'plan_name': None,
                    'planning_exam_name': None,
                    'planningCT_filename': None,
                    'evaluation_examinations': None
                })

        return plan_details

    def get_plan_details(self):
        return self.plan_details
    
# # Usage as an example
# file_path = '/home/shahpouriz/Data/DBP_oldDATA/nrrd/foton/results_DBP_HN025_20230705_220829.json'
file_path = '/home/shahpouriz/Data/DBP_oldDATA/results_DBP_HN002_20230630_155010.json'
treatment_data = RawDataReader (file_path)
# print(treatment_data.id)
plan_details = treatment_data.get_plan_details()

print(plan_details[0].get('plan_name'))
print(plan_details[0].get('planningCT_filename'))
print(plan_details[0].get('evaluation_examinations'))

A1PHH
DBP_HN002/A1PHH/rtdose_pCTp0_1.2.752.243.1.1.20230619155452641.7071.30386.dcm
[{'final_translation_coordinate': {'x': -0.14788, 'y': -0.15029, 'z': 0.09252}, 'repeatedCT_filename': 'DBP_HN002/A1PHH/rCTp1/optimization_case_0/rtdose_rCTp1_1.2.752.243.1.1.20230619155630476.6004.92857.dcm'}, {'final_translation_coordinate': {'x': 1.0208, 'y': 1.06197, 'z': -0.66347}, 'repeatedCT_filename': 'DBP_HN002/A1PHH/rCTp1/optimization_case_1/rtdose_rCTp1_1.2.752.243.1.1.20230619160146218.6015.92457.dcm'}, {'final_translation_coordinate': {'x': 0.78146, 'y': 1.34488, 'z': 1.41588}, 'repeatedCT_filename': 'DBP_HN002/A1PHH/rCTp1/optimization_case_2/rtdose_rCTp1_1.2.752.243.1.1.20230619160902203.7201.20505.dcm'}, {'final_translation_coordinate': {'x': -1.09572, 'y': -0.59783, 'z': -0.14909}, 'repeatedCT_filename': 'DBP_HN002/A1PHH/rCTp1/optimization_case_3/rtdose_rCTp1_1.2.752.243.1.1.20230619161651313.8108.19605.dcm'}, {'final_translation_coordinate': {'x': 0.99727, 'y': 1.29313, 'z': 1.43521}, '

--------------------

In [278]:
class DirectoryProcessor:
    def __init__(self, directory_path):
        self.directory_path = directory_path

    def process_files(self):
        file_pattern = os.path.join(self.directory_path, 'results_DBP_HN*.json')
        json_files = glob.glob(file_pattern)
        results = []
        for file_path in json_files:
            reader = RawDataReader(file_path)
            file_results = {
                'ID': reader.id,  # Ensure this is correctly capturing the 'ID' from each JSON
                'plan_details': reader.get_plan_details()
            }
            results.append(file_results)
        return results


# Usage
directory_path = '/home/shahpouriz/Data/DBP_oldDATA'
processor = DirectoryProcessor(directory_path)
all_results = processor.process_files()

for result in all_results:
    print(result)

{'ID': 'DBP_HN014', 'plan_details': [{'plan_name': 'A1PHH', 'planning_exam_name': 'pCTp0', 'planningCT_filename': 'DBP_HN014/A1PHH/rtdose_pCTp0_1.2.752.243.1.1.20230622121511492.2918.80068.dcm', 'evaluation_examinations': [{'final_translation_coordinate': {'x': -0.01182, 'y': 0.00716, 'z': -0.00033}, 'repeatedCT_filename': 'DBP_HN014/A1PHH/rCTp2/optimization_case_0/rtdose_rCTp2_1.2.752.243.1.1.20230622121808211.5511.94317.dcm'}, {'final_translation_coordinate': {'x': 0.63766, 'y': 0.10591, 'z': -0.79171}, 'repeatedCT_filename': 'DBP_HN014/A1PHH/rCTp2/optimization_case_1/rtdose_rCTp2_1.2.752.243.1.1.20230622122407999.9805.40942.dcm'}, {'final_translation_coordinate': {'x': 0.22822, 'y': 1.30796, 'z': 0.91983}, 'repeatedCT_filename': 'DBP_HN014/A1PHH/rCTp2/optimization_case_2/rtdose_rCTp2_1.2.752.243.1.1.20230622123335703.2349.30574.dcm'}, {'final_translation_coordinate': {'x': -1.0931, 'y': 0.24231, 'z': -0.69071}, 'repeatedCT_filename': 'DBP_HN014/A1PHH/rCTp2/optimization_case_3/rtdose

------------
# convert_dicom_to_nrrd

In [280]:
import os
import SimpleITK as sitk
import json  # Assuming you might need it for processing JSON data

class DICOMtoNRRDConverter:
    def __init__(self, directory_path, output_directory):
        self.directory_path = directory_path
        self.output_directory = output_directory
        self.directory_processor = DirectoryProcessor(directory_path)

    def convert_and_save(self):
        patient_data = self.directory_processor.process_files()

        for patient in patient_data:
            patient_id = patient['ID']
            patient_output_path = os.path.join(self.output_directory, patient_id)

            if not os.path.exists(patient_output_path):
                os.makedirs(patient_output_path)


            for plan_index, plan in enumerate(patient['plan_details']):
                
                plan_name = 'P'

                # Convert and update planning CT
                planning_ct_nrrd_name = self._generate_nrrd_filename(patient_id, plan_name, plan_index, "planningCT")
                nrrd_path = self._process_dose_file(plan['planningCT_filename'], patient_output_path, planning_ct_nrrd_name)

                plan['planningCT_filename'] = nrrd_path or plan['planningCT_filename']

        return patient_data

    def _generate_nrrd_filename(self, patient_id, plan_name, plan_index, exam_type, exam_index=None):
        filename = f"{patient_id}_{plan_name}{plan_index}_{exam_type}"
        if exam_index is not None:
            filename += f"{exam_index}"
        filename += ".nrrd"
        return filename

    def _process_dose_file(self, dicom_name, folder_path, nrrd_filename):
        dicom_path = os.path.join(self.directory_path, dicom_name)
        nrrd_path = os.path.join(folder_path, nrrd_filename)
        
        try:
            dose_img = sitk.ReadImage(dicom_path)
            sitk.WriteImage(dose_img, nrrd_path)
            print(f"Successfully created: {nrrd_path}")
            return nrrd_path  # Return the new NRRD filename
        except Exception as e:
            print(f"Error processing file {dicom_path}: {e}")
            return None

# Assuming DirectoryProcessor is correctly implemented to parse input data

directory_path = '/home/shahpouriz/Data/DBP_oldDATA'
output_directory_proton = '/home/shahpouriz/Data/DBP_oldDATA/nrrd/proton'
output_directory_foton = '/home/shahpouriz/Data/DBP_oldDATA/nrrd/foton'

converter = DICOMtoNRRDConverter(directory_path, output_directory_proton)
updated_patient_data = converter.convert_and_save()


/home/shahpouriz/Data/DBP_oldDATA/DBP_HN014/A1PHH/rtdose_pCTp0_1.2.752.243.1.1.20230622121511492.2918.80068.dcm
Successfully created: /home/shahpouriz/Data/DBP_oldDATA/nrrd/proton/DBP_HN014/DBP_HN014_P0_planningCT.nrrd
/home/shahpouriz/Data/DBP_oldDATA/DBP_HN014/A1PHH/rtdose_pCTp0_1.2.752.243.1.1.20230622121511492.2918.80068.dcm
Successfully created: /home/shahpouriz/Data/DBP_oldDATA/nrrd/proton/DBP_HN014/DBP_HN014_P1_planningCT.nrrd
/home/shahpouriz/Data/DBP_oldDATA/DBP_HN014/A1PHH/rtdose_pCTp0_1.2.752.243.1.1.20230622121511492.2918.80068.dcm
Successfully created: /home/shahpouriz/Data/DBP_oldDATA/nrrd/proton/DBP_HN014/DBP_HN014_P2_planningCT.nrrd
/home/shahpouriz/Data/DBP_oldDATA/DBP_HN014/A1PHH/rtdose_pCTp0_1.2.752.243.1.1.20230622121511492.2918.80068.dcm
Successfully created: /home/shahpouriz/Data/DBP_oldDATA/nrrd/proton/DBP_HN014/DBP_HN014_P3_planningCT.nrrd
/home/shahpouriz/Data/DBP_oldDATA/DBP_HN014/A1PHH/rtdose_pCTp0_1.2.752.243.1.1.20230622121511492.2918.80068.dcm
Successfully

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'NoneType'

------------
# convert_dicom_to_nrrd

In [ ]:

# Save updated_patient_data to a JSON file
output_json_file = directory_path + '/nrrd/nrrd_file_info.json' 
with open(output_json_file, 'w') as file:
    json.dump(updated_patient_data, file, indent=4)

print(f"Updated patient data saved to {output_json_file}")


Updated patient data saved to /home/shahpouriz/Data/DBP_oldDATA/nrrd/nrrd_file_info.json
